In [16]:
import numpy as np
import struct
import pandas as pd
import matplotlib.pyplot as plt
import dash
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objs as go
import json

def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

def add_frame_nr(df,i,k,amount_frames):

    while i < amount_frames:
        if i == 0:
            iloc_1 = 0
        if i > 0:
            iloc_1 = k - 2907
        iloc_2 = k
        df.loc[iloc_1:iloc_2,'frames'] = int(i)
        k += 2907
        i += 1
    return df

def read_pc2(path):
    with open(path, 'rb') as f:
        head_fmt = '<12siiffi'
        data_fmt = '<fff'
        head_unpack = struct.Struct(head_fmt).unpack_from
        data_unpack = struct.Struct(data_fmt).unpack_from
        data_size = struct.calcsize(data_fmt)
        headerStr = f.read(struct.calcsize(head_fmt))
        head = head_unpack(headerStr)
        nverts, nframes = head[2], head[5]
        data = []
        for i in range(nverts*nframes):
            data_line = f.read(data_size)
            if len(data_line) != data_size:
                return None
            data.append(list(data_unpack(data_line)))
        data = np.array(data).reshape([nframes, nverts, 3])
    arr_reshaped = data.reshape(data.shape[0]*data.shape[1], data.shape[2])
    df = pd.DataFrame(arr_reshaped)
    df.columns = ['x_val2','y_val2','z_val2']
    df = add_frame_nr(df,0,2907,df.shape[0]/2907)

    return df


df = read_pc2(r'C:\files\KLA0057Animation_stabilised.pc2')
df = df.iloc[0:2907]

############################################################################
#
#
## Here starts the Dash App
#
#
############################################################################


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})

# Creation of Figure
import plotly.graph_objects as go
def create_figure(skip_points=[]):
    dfs = df.drop(skip_points)
    fig = go.Figure(data =[go.Scatter3d(x = dfs['x_val2'],
                                   y = dfs['y_val2'],
                                   z = dfs['z_val2'],
                                   mode ='markers',
                                   marker = dict(
                                     color="blue",
                                     size = 5,
                                     opacity = 0.6,
                                    line=dict(
                                        color='black',
                                        width=1),
                                    

                                   )
)]
    

    )
    fig.update_layout(uirevision='wrong')
    return fig
f = create_figure()


# Define HTML Layout
app.layout = html.Div(
                    [html.Button('Delete', id='delete'),
                    html.Button('Clear Selection', id='clear'),
                    html.Button('Procrustes analysis', id='procrustes_analyses'),
                    dcc.Graph(id = '3d_scat', figure=f,style={'width': '50%','height':'1080px','padding-left':'25%', 'padding-right':'25%'}),
                    html.Div('selected:'),
                    html.Div(id='selected_points'), #, style={'display': 'none'})),
                    html.Div('deleted:'),
                    html.Div(id='deleted_points'),
                    html.Div('Procrustes:'),
                    html.Div(id='body-div')
])

# Function to delete Selected Points:
@app.callback(
            Output('deleted_points', 'children'),
            Input('delete', 'n_clicks'),
            State('selected_points', 'children'),
            State('deleted_points', 'children')
 )

def delete_points(n_clicks, selected_points, delete_points):
    print('n_clicks:',n_clicks)
    if selected_points:
        selected_points = json.loads(selected_points)
    else:
        selected_points = []

    if delete_points:
        deleted_points = json.loads(delete_points)
    else:
        deleted_points = []
    ns = [p['pointNumber'] for p in selected_points]
    new_indices = [df.index[n] for n in ns if df.index[n] not in deleted_points]
    print('new',new_indices)
    deleted_points.extend(new_indices)
    return json.dumps(deleted_points)

# Funciton to select the Datapoint into a List
    
@app.callback(
        Output('selected_points', 'children'),
        Input('3d_scat', 'clickData'),
        Input('deleted_points', 'children'),
        Input('clear', 'n_clicks'),
        State('selected_points', 'children'))



def select_point(clickData, deleted_points, clear_clicked, selected_points):
    ctx = dash.callback_context
    ids = [c['prop_id'] for c in ctx.triggered]

    if selected_points:
        results = json.loads(selected_points)
        #print(selected_points)
        my_array = np.asarray(results)
        print(my_array)
    else:
        results = []


    if '3d_scat.clickData' in ids:
        if clickData:
            for p in clickData['points']:
                if p not in results:
                    results.append(p)
    if 'deleted_points.children' in ids or  'clear.n_clicks' in ids:
        results = []
    results = json.dumps(results)
    return results


# Function to delete Selected Points:
@app.callback(
            Output('body-div', 'children'),
            Input('procrustes_analyses', 'n_clicks'),
            State('selected_points', 'children'),
            State('body-div', 'children')
 )



def procrustes_points(n_clicks, selected_points, delete_points):
    print('n_clicks:',n_clicks)
    y = json.loads(selected_points)
    k=0
    for i in y:
        data = {'x':i["x"], 'y':i["y"], 'z':i["z"]}
        print(i["x"],i["y"],i["z"])
        if k == 0:
            df_procrustes_1 = pd.DataFrame(data, index=[k])
            k = k+1
        else:
            df_procrustes_1 = df_procrustes_1.append(data,ignore_index=True)
            k = k+1

    A = df_procrustes_1.to_numpy()
    

    ########################################################################
    #
    #
    #Procrustes Test Start
    #
    #
    #########################################################################

    # Random rotation and translation
    R = np.random.rand(3,3)
    t = np.random.rand(3,1)

    # make R a proper rotation matrix, force orthonormal
    U, S, Vt= np.linalg.svd(R)
    R = U@Vt

    # remove reflection
    if np.linalg.det(R) < 0:
        Vt[2,:] *= -1
        R = U@Vt

    # number of points
    n = 8

    #A = np.random.rand(3, n)
    B = R@A + t

    # Recover R and t
    ret_R, ret_t = rigid_transform_3D(A, B)

    # Compare the recovered R and t with the original
    B2 = (ret_R@A) + ret_t

    # Find the root mean squared error
    err = B2 - B
    err = err * err
    err = np.sum(err)
    rmse = np.sqrt(err/n)

    print("Points A")
    print(A)
    print("")

    print("Points B")
    print(B)
    print("")

    print("Ground truth rotation")
    print(R)

    print("Recovered rotation")
    print(ret_R)
    print("")

    print("Ground truth translation")
    print(t)

    print("Recovered translation")
    print(ret_t)
    print("")

    print("RMSE:", rmse)

    if rmse < 1e-5:
        print("Everything looks good!")
    else:
        print("Hmm something doesn't look right ...")

    ########################################################################
    #
    #
    #Procrustes Test End
    #
    #
    #########################################################################

    return str(A)


#Function to create Figure:
@app.callback(
            Output('3d_scat', 'figure'),
            Input('selected_points', 'children'),
            Input('deleted_points', 'children'),
            State('deleted_points', 'children'))

def chart_3d( selected_points, deleted_points_input, deleted_points_state):
    global f
    deleted_points = json.loads(deleted_points_state) if deleted_points_state else []
    f = create_figure(deleted_points)

    selected_points = json.loads(selected_points) if selected_points else []
    if selected_points:
        f.add_trace(
            go.Scatter3d(
                mode='markers',
                x=[p['x'] for p in selected_points],
                y=[p['y'] for p in selected_points],
                z=[p['z'] for p in selected_points],
                marker=dict(
                    color='white',
                    size=18,
                    line=dict(
                        color='red',
                        width=2
                    )
                ),
                showlegend=False
            )
        )
    return f




if __name__ == '__main__':
    app.run_server(debug=False, port=8044)

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

Dash is running on http://127.0.0.1:8044/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8044/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 20:42:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:42:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:42:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:42:16] "POST /_dash-update-component HTTP/1.1" 200 -


n_clicks: None
new []
n_clicks: None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\mhoeger\Anaconda3\lib\site-packages\dash\

127.0.0.1 - - [10/Dec/2021 20:42:16] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [10/Dec/2021 20:42:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:42:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:42:40] "POST /_dash-update-component HTTP/1.1" 200 -


[]


...

[{'x': 25.85047721862793, 'y': -31.699462890625, 'z': -23.054187774658203, 'curveNumber': 0, 'pointNumber': 1298}]


127.0.0.1 - - [10/Dec/2021 20:42:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:43:06] "POST /_dash-update-component HTTP/1.1" 200 -


[{'x': 25.85047721862793, 'y': -31.699462890625, 'z': -23.054187774658203, 'curveNumber': 0, 'pointNumber': 1298}
 {'x': -22.252670288085938, 'y': -39.07358932495117, 'z': -23.211578369140625, 'curveNumber': 0, 'pointNumber': 1918}]


127.0.0.1 - - [10/Dec/2021 20:43:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:43:16] "POST /_dash-update-component HTTP/1.1" 200 -


n_clicks: 1
25.85047721862793 -31.699462890625 -23.054187774658203
-22.252670288085938 -39.07358932495117 -23.211578369140625
2.955420970916748 -64.01475524902344 2.672417163848877
det(R) < R, reflection detected!, correcting for it ...
Points A
[[ 25.85047722 -31.69946289 -23.05418777]
 [-22.25267029 -39.07358932 -23.21157837]
 [  2.95542097 -64.01475525   2.67241716]]

Points B
[[-16.31280632 -40.09944586  11.21667794]
 [ 26.89286016 -53.55535141 -10.37509022]
 [-13.21001758 -45.49351897 -28.89100049]]

Ground truth rotation
[[-0.57136195  0.19520679  0.7971448 ]
 [ 0.76309213 -0.23111925  0.60355139]
 [ 0.30205284  0.95314123 -0.01690802]]
Recovered rotation
[[-0.57136195  0.19520679  0.7971448 ]
 [ 0.76309213 -0.23111925  0.60355139]
 [ 0.30205284  0.95314123 -0.01690802]]

Ground truth translation
[[0.44514661]
 [0.23979532]
 [0.24168005]]
Recovered translation
[[0.44514661]
 [0.23979532]
 [0.24168005]]

RMSE: 1.0527818253729124e-14
Everything looks good!
